# Lab Sheet 6: Movie recommendation using ALS in Spark

These tasks are for working in the lab session and during the week. We will use Alternating Least Squares (ALS) in Spark to recommend movies and explore the outcome of a cross-validation to find its optimal parameter settings.

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

In [ ]:
#install spark
%cd
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!tar -xzf "/content/drive/My Drive/Big_Data/data/spark/spark-3.5.0-bin-hadoop3.tgz"
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/root/spark-3.5.0-bin-hadoop3"
%cd /content
import findspark
findspark.init()

import pyspark
# get a spark context
sc = pyspark.SparkContext.getOrCreate()
print(sc)
# and a spark session
spark = pyspark.sql.SparkSession.builder.getOrCreate()
print(spark)

## Task 1) Read the data into a DataFrame

We will use a dataset of movie ratings, consisting of four columns separated by `::` A sample is displayed below. The first column contains `userId`s, the second `movieId`s, the third the `rating` of the user for that movie and the last column is a `timestamp`.

In [ ]:
!head -n 5 "/content/drive/MyDrive/Big_Data/data/movielens-small/sample_movielens_ratings.txt"

**Read** the data, split into **tokens** and create a **structured DataFrame**. For low level tasks like splitting strings, we use an RDD where we can apply a `map` function.

See here for how to create a **new Row object**:
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Row.html
When creating a new Row object, it is helpful to ensure that the elements in the row have a specific type. You can achive this by casting/converting a variable to a type, e.g. with int() https://docs.python.org/3.7/library/functions.html#int or float() https://docs.python.org/3.7/library/functions.html#float or str() https://docs.python.org/3.7/library/stdtypes.html#str .

and here for **random split**ting of a DataFrame
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.randomSplit.html

An 80%-20% train/test set split is common.

In [ ]:
from pyspark.sql import Row
# the imports are used for creating the data frame

# this gets us an RDD. (could also be done with RDD.textFile in this case)
lines = spark.read.text("/content/drive/MyDrive/Big_Data/data/movielens-small/sample_movielens_ratings.txt").rdd
# the RDD items now look like this: Row(value="1::2::3::123465769")
#now split the lines at the '::'
>>>parts = lines.map(lambda row: row.value ... ) # <<< split the row value using Python's split function for strings
>>>ratingsRDD = parts.map( ... ) # <<< create a new Row object with userId as int, movieId  as int, rating as float, timestamp as int
ratings = spark.createDataFrame( ratingsRDD ) # create a dataframe from an RDD of Row objects
ratings.createOrReplaceTempView('ratings') # register the DataFrame so that we can use it with Spark SQL.
>>>(training, test) =  ... #<<< create a random split into test and training set from the dataframe
print(training.show(5)) # just for testing, should show five columns
print(training.count()) # just for testing, should be around 1200

## Task 2) Create a baseline recommendation

Now take a very **simple estimate** as the baseline: calculate the **mean of all ratings**.    

The average can be calculated with the **SQL `AVG` command**, within an SQL `SELECT` statement. If you replace selected column, e.g. `rating`, with `AVG(rating)`, the returned DataFrame will contain only 1 row. You can then access the contents of the row by its name, here as `row['avg(rating)']` (avg needs to be lower case here).

Then calculate the **squared error** with respect to the average as predictor. You can again use the SQL `AVG` command on the error for the mean of the squared errors.


In [ ]:
# select the average of the 'rating' entries from 'ratings' DF
>>>SQL1 = ' ... ' #<<<
row = spark.sql(SQL1).collect()[0] # get the single row with the result
print('row', row)
>>>meanRating = ... # access Row as a dictionary
print('meanRating', meanRating)

>>>se_rdd = test.rdd.map( ... ) #<<< get the the squared error (difference to the average) using Python pow()
se_df = spark.createDataFrame(se_rdd) # create a data frame
>>>se_df.createOrReplaceTempView( ... ) #<<< Register with the SQL system (choose a name)
print('se_df', se_df)

# get the average squared error
>>>SQL2 = '...' #<<< Use the name here that you chose 4 lines above
row = spark.sql(SQL2).collect()[0]
>>>meanSE = ... #<<< access Row as a dictionary
from math import sqrt
>>>print('RMSE', ...) #<<< calculate the root of the meanSE using Python's sqrt function

## Task 3) Find the best parameters for ALS by cross-validation on the training set

Now **create** an **ALS estimator** and a **parameter grid** to explore different values for the `rank` and `regParam` parameter of the ALS. Then build a **cross-validator** to train the model and optimise the parameters.

Here is the doc for the **ALS class**:
https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.recommendation.ALS.html#pyspark.ml.recommendation.ALS

Here it is for the **ParamGridBuilder**:
https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.tuning.ParamGridBuilder.html#pyspark.ml.tuning.ParamGridBuilder

See here for more information about the **RegressionEvaluator**:
https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.RegressionEvaluator.html#pyspark.ml.evaluation.RegressionEvaluator

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, rank=5, regParam=0.1, userCol="userId", itemCol="movieId", ratingCol="rating")

# build a parameter grid for rank and regParam.
>>>paramGrid = ParamGridBuilder() \
    .addGrid(...) \
    .addGrid(...).build() # <<<< For a first test, use just one value per parameter

# set up a regression evaluater evaluating RMSE
>>>regEval = RegressionEvaluator( ... ) # <<<<

# set up a cross validator with the als, paramGrid and regEval
>>>crossVal = CrossValidator( ... numFolds=3) # <<<<

print('starting cross-validation')
cvModel = crossVal.fit(training)
print('finished cross-validation')

## Task 4) Evaluate the best ALS model

Take the trained cvModel and **extract the best parameter values** by inspecting the estimatorParameterMap. **Compare** the RMSE value of the trained model for different parameter settings to that of the mean.

The parameter maps and metrics lists we get from the `cvModel` are local Python list, so we need to use standard Python methods, not RDD methods. There are however similar functions available, in particular `map` and `zip`, which work like for RDDs and `list` which is similar to RDD.collect in creating a mapped list. See here for documentation:  [https://docs.python.org/3/library/functions.html](https://docs.python.org/3/library/functions.html)

In [ ]:
print('parameter combinations: ', cvModel.getEstimatorParamMaps()) # get parameter combinations
print('metrics: ', cvModel.avgMetrics) # the metrics from the CrossValidation
# use Python zip and list (not RDD functions, these are local Python objects)
# to create a joint parameter and result list
>>>paramMap = ... #<<<
print(paramMap) # for testing
print('\n') # for readability of the output
# use Python min to get the best params (i.e. those producing minimal RMSE)
paramMax = min(paramMap, key=lambda x: x[1])
print('optimal parameters (giving lowest RMSE): ', paramMax)
# now we have the best parameters and the best test value from CV

**Evaluate** the `cvModel` by computing the RMSE on the held-out test data

In [ ]:
>>>test_pred = ... #<<< transform with the cvModel
rmse = regEval.evaluate(test_pred)
print("Root-mean-square error = " + str(rmse))

## Task 5) Scale up to Google Cloud

**Apply** the approach above to the **larger MovieLens dataset** (or part of it). The data is available at `Big_Data/data/movielens-small/ratings.csv`.
It is available in CSV form, so that you will need to **adapt the code** to read that data.

**Extract a Python file** from the code in this notebook, and apply that in the cloud (you can use the [`sparkSubmitDemo.py` example on Moodle](https://moodle4.city.ac.uk/mod/resource/view.php?id=381975) as a starting point). To upload the data to Google Storage, you can follow the [tutorial on Moodle](https://moodle4.city.ac.uk/pluginfile.php/441245/mod_resource/content/1/Google%20Cloud%20Intro%202024.pdf).